This analysis was requested to help inform some discussions about article creation dynamics on the English Wikipedia. See [T149021](https://phabricator.wikimedia.org/T149021) and [T149049](https://phabricator.wikimedia.org/T149049).

In [1]:
import pandas as pd
import bokeh.plotting as bk
import bokeh
import datetime as dt
from IPython.display import display, HTML

In [7]:
#!/usr/bin/python
import pymysql
import pandas as pd
from impala.dbapi import connect as impala_conn
from impala.util import as_pandas

def try_decode(cell):
    try:
        return cell.decode(encoding = "utf-8")
    except AttributeError:
        return cell
    
def decode_data(d):
    return [{try_decode(key): try_decode(val) for key, val in item.items()} for item in d]

def query_db(query, db = "mariadb", fmt = "pandas"):
    if db not in ["mariadb", "hadoop"]:
        raise ValueError("The db should be `mariadb` or `hadoop`.")
    if fmt not in ["pandas", "raw"]:
        raise ValueError("The format should be either `pandas` or `raw`.")
    
    if db == "mariadb":
        try:
            conn = pymysql.connect(
                host = "analytics-store.eqiad.wmnet",
                read_default_file = '/etc/mysql/conf.d/research-client.cnf',
                charset = 'utf8mb4',
                db='staging',
                cursorclass=pymysql.cursors.DictCursor
            )
            if fmt == "pandas":
                result = pd.read_sql_query(query, conn)
                # Turn any binary data into strings
                result = result.applymap(try_decode)
            elif fmt == "raw":
                cursor = conn.cursor()
                cursor.execute(query)
                result = cursor.fetchall()
                result = decode_data(result)
        finally:
            conn.close()
        
    elif db == "hadoop":
        try:
            hive_conn = impala_conn(host='analytics1003.eqiad.wmnet', port=10000, auth_mechanism='PLAIN')
            hive_cursor = hive_conn.cursor()
            hive_cursor.execute(query)
            if fmt == "pandas":
                result = as_pandas(hive_cursor)
            elif fmt == "raw":
                result = hive_cursor.fetchall()
        finally:
            hive_conn.close()
    
    return result

# Survival of new articles over time

Data from the following queries (surviving creations are held in the `revision` table, while deleted creations have been moved to the `archive` table):

```
select left(rev_timestamp, 6) as month, count(*) as surviving_creations
from enwiki.revision
left join enwiki.page 
on rev_page = page_id
where 
page_namespace = 0 and
rev_parent_id = 0 and
convert(rev_comment using utf8) not like "%redir%" and
rev_len > 100
group by left(rev_timestamp, 6);
```

```
select left(a.ar_timestamp, 6) as month, count(*) as deleted_creations
from enwiki.archive a
inner join
(
select ar_title, min(ar_timestamp) as ar_timestamp
from enwiki.archive
where
ar_namespace = 0 and
convert(ar_comment using utf8) not like "%redir%" and
ar_len > 100
group by ar_title
) b
using (ar_title, ar_timestamp)
group by left(a.ar_timestamp, 6)
```

In [2]:
survived = pd.read_table("2016-10_enwiki_surviving_creations.tsv")
survived.head()

,month,surviving_creations
0,200101,156
1,200102,327
2,200103,531
3,200104,582
4,200105,1070


In [3]:
deleted = pd.read_table("2016-10_enwiki_deleted_creations.tsv")
deleted.head()

,month,deleted_creations
0,200101,15
1,200102,46
2,200103,35
3,200104,15
4,200105,53


In [4]:
survival = survived.merge(deleted, on = "month")
survival["pct_survival"] = \
    survival["surviving_creations"] / \
    (survival["surviving_creations"] + survival["deleted_creations"])

# Convert month column to real date
survival["month"] = pd.to_datetime(survival["month"], format = "%Y%m")
survival.set_index(keys = "month", inplace = True)

# Get rid of incomplete data for November
survival.drop(pd.to_datetime("2016-11-01"), inplace = True)

survival.tail()

,surviving_creations,deleted_creations,pct_survival
month,,,
2016-06-01,21796,8298,0.724264
2016-07-01,22971,7817,0.746102
2016-08-01,23873,7999,0.749027
2016-09-01,26488,7998,0.768080
2016-10-01,24203,6978,0.776210


In [5]:
bokeh.io.output_notebook()

c = bk.figure(width = 800, height = 400, x_axis_type = "datetime", y_range = (0, 1))
c.line(survival.index, survival["pct_survival"], color = "navy", line_width = 2)
c.toolbar.active_drag = None
bk.show(c)

Loading BokehJS ...

# Article creation by non-autoconfirmed editors

## Data Lake

It should be possible to get this data from the [Data Lake](https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake), which would make it possible to include data on deleted articles (I previously tried this using the MariaDB replicas, but it was unable to handle the complexity of the query).

In [13]:
dl_creations_hql = """
select
mediawiki_history.wiki_db,
substr(event_timestamp, 0, 8) as day,
count(*) as creations,
sum(
        if(
            ((unix_timestamp(event_timestamp, 'yyyyMMddHHmmss') -
            unix_timestamp(coalesce(event_user_creation_timestamp, '20050101000000'), 'yyyyMMddHHmmss'))
            < 345600) or
            (user_edits_by_hour.edit_count < 10),
        1, 0)
    )
  as non_autoconfirmed_creations

from wmf.mediawiki_history
       inner join
   (select wiki_db,
           event_user_id,
           hour,
           sum(edit_count) over (partition by wiki_db, event_user_id order by hour) as edit_count
      from (select wiki_db,
                   event_user_id,
                   substr(event_timestamp, 0, 10) as hour,
                   count(*) as edit_count
              from wmf.mediawiki_history
             where event_entity = 'revision'
               and event_type = 'create'
               and wiki_db = 'enwiki'
               and snapshot = '2017-04'
             group by wiki_db, event_user_id, substr(event_timestamp, 0, 10)
           ) user_edits_per_hour
   ) user_edits_by_hour    ON mediawiki_history.wiki_db = user_edits_by_hour.wiki_db
                          AND mediawiki_history.event_user_id = user_edits_by_hour.event_user_id
                          AND substr(event_timestamp, 0, 10) = user_edits_by_hour.hour
where event_entity = 'revision'
  and event_type = 'create'
  and page_namespace = 0
  and mediawiki_history.wiki_db = 'enwiki'
  and revision_parent_id = 0
  and event_comment not regexp "[Rr]edir"
  and snapshot = '2017-04'
  and event_timestamp > '20170101000000'

 group by mediawiki_history.wiki_db, substr(event_timestamp, 0, 8)
 order by mediawiki_history.wiki_db, day
 limit 100000
"""

In [ ]:
dl_creations = query_db(dl_creations_hql, db = "hadoop")

## Recent changes

**Summary**: I estimate that about 87% of new articles on the English Wikipedia  are created by autoconfirmed users. (In this case, an article is a main-namespace page which is not a redirect.)

This estimate is based on articles created in the week before the query was run, and does *not* include any articles created and then deleted by that time (anywhere from 0 to 7 days after their creation).

In [ ]:
page_creations = """
select 
    page_title,
    page_title_latest,
    page_id,
    event_timestamp,
    revision_id as creation_rev_id,
    revision_text_bytes as length_at_creation,
    event_user_id as creator_id,
    event_user_text as creator_name,
    user_creator_timestamp as creator_registration
from wmf.mediawiki_history
where 
    wiki_db = "enwiki" and
    event_entity = "revision" and
    event_type = "create"
    revision_parent_id = 0
    page_namespace = 0 and
    event_comment not regexp "[Rr]edir" and
    event_timestamp >= "201704" and
    event_timestamp < "201705"
    ;
"""

In [45]:
creations = pd.read_table(
    "2016-10_enwiki_article_creations.tsv",
    parse_dates = [2, 8])
creations.head()

,page_title,page_id,creation_timestamp,creation_rev_id,length_at_creation,user_id,user_name,user_edit_count,user_registration
0,Sundkler,52087925,2016-10-25 00:00:56,746047690,28,29493069,NorbayWarte,36,2016-10-24 21:14:31
1,Donald_Trump's_Wall,52087945,2016-10-25 00:04:02,746048100,1043,29133239,Christianhamby,1,2016-09-08 20:39:24
2,Bye_Bye_My_Blue,52088044,2016-10-25 00:21:22,746050319,1055,28768139,LuckyAries,690,2016-07-15 02:02:37
3,Ahmad_Danny_Ramadan,52088127,2016-10-25 00:31:26,746051791,1430,12755007,Danny3aw,0,2010-07-19 06:26:26
4,ACB_statistical_leaders,52088148,2016-10-25 00:35:56,746052392,7791,13174094,Bluesangrel,56083,2010-10-01 17:40:10


In [46]:
four_d = dt.timedelta(days = 4)
creations["creator_autoconfirmed"] = (
    (creations["user_edit_count"] >= 10) &
    (creations["creation_timestamp"] >= (creations["user_registration"] + four_d))
    )

However, this leaves a couple entries with null account creations dates because their accounts were created before Mediawiki started recording them. I'll manually set them to be autoconfirmed.

In [47]:
null_reg = creations[ creations["user_registration"].isnull() ]
null_reg

,page_title,page_id,creation_timestamp,creation_rev_id,length_at_creation,user_id,user_name,user_edit_count,user_registration,creator_autoconfirmed
1053,Stone_slab,52134207,2016-10-29 20:22:38,746827427,6979,44656,Mcapdevila,3787,NaT,False
1335,Niccolò_Lorini,52149965,2016-10-31 16:27:31,747128371,453,36571,Acrider,116,NaT,False
5305,Takashi_Nishimoto,52141654,2016-10-30 17:42:34,746968857,2149,602857,Muboshgu,173629,NaT,False


In [48]:
creations.ix[null_reg.index, "creator_autoconfirmed"] = True
creations[ creations["user_registration"].isnull() ]["creator_autoconfirmed"]

1053    True
1335    True
5305    True
Name: creator_autoconfirmed, dtype: bool

In [49]:
creations.groupby("creator_autoconfirmed").size()

creator_autoconfirmed
False     735
True     5477
dtype: int64

So 88.1% of creations were by autoconfirmed users. But I think there are still a good number of redirect creations here, even though I filtered out most of them using the edit summary. What if we pull out everything where the inital size was less than 100 bytes and the user was autoconfirmed? From spot-checking, it looks like that should get most of them while not removing too many creations of real stubs.

In [50]:
to_remove = creations[
    (creations["length_at_creation"] < 100) & 
    (creations["creator_autoconfirmed"] == True)
]

creations = creations.drop(to_remove.index)

In [51]:
creations.groupby("creator_autoconfirmed").size()

creator_autoconfirmed
False     735
True     4876
dtype: int64

That gives 86.9% of creations by autoconfirmed users. That's likely a better estimate, though it's not a large difference in any case. 

Ideally, I'd check the text of each revision to know for sure whether it was a redirect at the time of creation. But that would require a lot of API work, and this suggests that it wouldn't change the results much.

## Example articles

I'll pull out a linked list of the creations from 31 October.

In [52]:
examples = creations[
    (creations["creation_timestamp"] >= "2016-10-31") &
    (creations["creation_timestamp"] < "2016-11-01")
]

ac_ex = examples[examples["creator_autoconfirmed"] == True]
non_ac_ex = examples[examples["creator_autoconfirmed"] == False]

def print_table(df):
    print("Printing {} rows".format(df.shape[0]))
    output = "<table><tr><th>Page</th><th>Initial version</th></tr>"
    for row in df.iterrows():
        table_row = """
            <tr>
                <td><a href='https://en.wikipedia.org/wiki/{title}'>{title}</a></td>
                <td><a href='http://en.wikipedia.org/wiki/Special:Diff/{rev_id}'>Special:Diff/{rev_id}</a></td>
            </tr>
            """
        output += table_row.format(title = row[1][0], rev_id = row[1][3])
    
    output += "</table>"

    display(HTML(output))

### Autoconfirmed creations

In [54]:
print_table(ac_ex)

Printing 695 rows


Page,Initial version
Naristae,Special:Diff/747021309
Linda_Aranaydo,Special:Diff/747044839
Almaty_Central_Mosque,Special:Diff/747046267
Central_Mosque_Almaty,Special:Diff/747047315
Princess_Agents,Special:Diff/747047374
Yong_Muhajil,Special:Diff/747052051
Western_Girls_(song),Special:Diff/747052847
Computer_Center_Corporation,Special:Diff/747054584
Soumahoro_Bangaly,Special:Diff/747055775
Ana-Patricia_Torea,Special:Diff/747057221


### Non-autoconfirmed creations


In [55]:
print_table(non_ac_ex)

Printing 104 rows


Page,Initial version
List_of_Hellevator_episodes,Special:Diff/747044382
Modern_Combat_6:_Versus,Special:Diff/747049425
Thai_fabrics_(Thai_woven_fabrics),Special:Diff/747054482
PEAT_(Progressive_Environmental_And_Agricultural_Technologies),Special:Diff/747054979
Rajindar_Nath_Rehbar,Special:Diff/747067637
Saman_Moghadam,Special:Diff/747067643
Prince_Dreambert,Special:Diff/747068052
Remberto_G._Sotto,Special:Diff/747068736
Products_from_Tanintharyi,Special:Diff/747070293
Rajiv_Krishna_Saxena,Special:Diff/747070407
